In [5]:
def calculate_score(self, ref_text, correct_score, score_lines):
        # Print the reference text for debugging
        print("Reference Text:", ref_text)

        # Calculate the index for scoring
        num_lines = len(ref_text)  # Number of lines in ref_text
        if num_lines > 0:
            scoring_index = min(len(score_lines) - 1, correct_score // (num_lines // len(score_lines)))
        else:
            scoring_index = 0
        
        # Handle case where the scoring index is out of range
        result = score_lines[scoring_index]
        
        # Print results
        print("Scoring Index:", scoring_index)
        print("Scoring Index Value:", result)
        print(f"Your score will be 20/{result}")

        return result
# Test the function
ref_text = ['96824', '84846', '43242', '42633']
correct_score = 5
score_lines = [80,60,40,20]
calculate_score(None, ref_text, correct_score, score_lines)

Reference Text: ['96824', '84846', '43242', '42633']
Scoring Index: 3
Scoring Index Value: 20
Your score will be 20/20


20

In [1]:
import easyocr  # type: ignore
import cv2  # type: ignore


class Image_processing:

    def __init__(self):
        self.repeat_count = 0

    def pre_process(self, image):
        """Pre-process the image for better OCR results."""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Increase contrast
        gray = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)
        # Apply adaptive thresholding
        processed = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
        )
        return processed

    def split_digits(self, ocr_result):
        """Split OCR results into single digits."""
        bbox, number, _ = ocr_result
        return [(bbox, digit) for digit in number if digit.isdigit()]

    def is_on_same_line(self, bbox1, bbox2, threshold=40):
        """Check if two bounding boxes are on the same line."""
        y1_center = (bbox1[0][1] + bbox1[2][1]) / 2
        y2_center = (bbox2[0][1] + bbox2[2][1]) / 2
        return abs(y1_center - y2_center) <= threshold

    def process_ocr_results(self, ocr_results, previous_texts, repeat_threshold=3):
        """Process OCR results to get unique numbers in lines."""
        all_digits = [
            item for result in ocr_results for item in self.split_digits(result)
        ]

        line_groups = []
        for bbox1, digit1 in all_digits:
            added_to_line = False
            for line in line_groups:
                if any(self.is_on_same_line(bbox1, bbox2) for bbox2, _ in line):
                    line.append((bbox1, digit1))
                    added_to_line = True
                    break
            if not added_to_line:
                line_groups.append([(bbox1, digit1)])

        lines = ["".join(digit for _, digit in line) for line in line_groups]

        if lines == previous_texts:
            self.repeat_count += 1
            if self.repeat_count >= repeat_threshold and lines:
                return None
        else:
            self.repeat_count = 0

        return lines

    def extract_scoring_index(self, line):
        """Extract scoring index from a line of numbers."""
        zero_index = line.find("0")
        if zero_index != -1:
            score = line[zero_index + 1 :]
            if score:
                return f"20/{line[:zero_index]}"
        return None

    def return_ocr_result(self, img):
        self.repeat_count = 0

        # Read the image
        img_path = img
        frame = cv2.imread(img_path)

        if frame is None:
            print("Error: Image not found or cannot be loaded.")
            return None, None

        # Zoom the image for better OCR results
        scale_factor = 2
        frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

        processed_frame = self.pre_process(frame)

        # Instance text detection
        reader = easyocr.Reader(["en"], gpu=True)

        threshold = 0.2
        previous_texts = []
        actual_output = []

        # try:
        text_raw = reader.readtext(processed_frame)
        current_results = [
            (bbox, text, score) for bbox, text, score in text_raw if score > threshold
        ]

        output = self.process_ocr_results(current_results, previous_texts)
        actual_output.append(output)
        if output is None:
            print("Repeated results. Breaking loop.")
            return None, None

        # Draw bounding boxes and texts
        for bbox, text, score in current_results:
            if score > threshold:
                top_left = tuple(map(int, bbox[0]))
                bottom_right = tuple(map(int, bbox[2]))
                if (
                    len(top_left) == 2 and len(bottom_right) == 2
                ):  # Ensure valid coordinates
                    cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 5)
                    cv2.putText(
                        frame,
                        text,
                        top_left,
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (255, 0, 0),
                        2,
                    )

        previous_texts = output
        print("Processed OCR Output:", output)

        # Display the image with drawn rectangles and text
        cv2.imshow("Text Recognition", frame)
        # cv2.waitKey(0)  # Wait indefinitely until a key is pressed
        cv2.destroyAllWindows()

        # except Exception as e:
        #     print("ERROR OCCUR", e)
        #     return None, None

        final_output = []
        scoring_index = []

        # Process final output to extract scoring index
        print("Output: ", output)

        # Output:  ['6096824', '0084846', '443242', '3042633']
        # Result should be

        # Scoring Index: ['20/60', '20/84', '20/44', '20/30']
        # Final Output: ['96824', '84846', '43242', '42633']
        final_output = []
        scoring_index = []

        # Process the output
        for line in output:
            temp_output = []
            temp_score = []

            line_length = len(line)

            for j in range(1, line_length):
                temp_score = line[:j]
                if line.endswith(temp_score):
                    temp_output = line[j : line_length - j]   
                    break

            final_output.append("".join(temp_output))
            scoring_index.append("".join(temp_score))
        # Convert to dictionary
        result_dict = {out: score for out, score in zip(final_output, scoring_index)}

        print("Final Output:", final_output)
        print("Scoring Index:", scoring_index)
        print("Result Dictionary:", result_dict)

        return final_output, scoring_index


image_file_path = [
    # "src/test_pictures/1st_pic.jpg",
    # "src/test_pictures/2nd_pic.jpg",
    # "src/test_pictures/3rd_pic.jpg",
    "src/test_pictures/4th_pic.jpg",
    "src/test_pictures/5th_pic.jpg"
    # ,
]


# Test the function
pic = "test.jpg"
Testing = Image_processing()

for pic in image_file_path:
    result_append, scoring = Testing.return_ocr_result(pic)


Using CPU. Note: This module is much faster with a GPU.


Processed OCR Output: ['60968260', '508484650', '404324240', '304268330']
Output:  ['60968260', '508484650', '404324240', '304268330']
Final Output: ['9682', '84846', '43242', '42683']
Scoring Index: ['60', '50', '40', '30']
Result Dictionary: {'9682': '60', '84846': '50', '43242': '40', '42683': '30'}


Using CPU. Note: This module is much faster with a GPU.


Processed OCR Output: ['2525', '78654', '2020', '32549', '1515', '56239', '1010', '82394']
Output:  ['2525', '78654', '2020', '32549', '1515', '56239', '1010', '82394']
Final Output: ['', '', '', '', '', '', '', '']
Scoring Index: ['25', '7865', '20', '3254', '15', '5623', '10', '8239']
Result Dictionary: {'': '8239'}
